In [4]:
from google.colab import files

uploaded = files.upload()


Saving match_score_dataset_soccer.csv to match_score_dataset_soccer.csv


In [5]:
import pandas as pd

file_path = "match_score_dataset_soccer.csv"
df = pd.read_csv(file_path)

df.head()


,Year,Score 1,Score 2,Player 1,Player 2,Winner
0,1930,France,Mexico,4,1,France
1,1930,USA,Belgium,3,0,USA
2,1930,Yugoslavia,Brazil,2,1,Yugoslavia
3,1930,Romania,Peru,3,1,Romania
4,1930,Argentina,France,1,0,Argentina


In [16]:
print("ستون‌های دیتاست:", df.columns)

print("\nمقدارهای یکتا در 'Team 1':", df["Team 1"].unique()[:10])
print("\nمقدارهای یکتا در 'Team 2':", df["Team 2"].unique()[:10])
print("\nمقدارهای یکتا در 'Winner':", df["Winner"].unique()[:10])


ستون‌های دیتاست: Index(['Year', 'Team 1', 'Team 2', 'Score 1', 'Score 2', 'Winner',
       'Team 1 Encoded'],
      dtype='object')

مقدارهای یکتا در 'Team 1': ['France' 'USA' 'Yugoslavia' 'Romania' 'Argentina' 'Chile' 'Uruguay'
 'Brazil' 'Paraguay' 'Austria']

مقدارهای یکتا در 'Team 2': ['Mexico' 'Belgium' 'Brazil' 'Peru' 'France' 'Bolivia' 'Paraguay'
 'Romania' 'Chile' 'USA']

مقدارهای یکتا در 'Winner': ['France' 'USA' 'Yugoslavia' 'Romania' 'Argentina' 'Chile' 'Uruguay'
 'Brazil' 'Paraguay' 'Austria']


In [19]:
print("تعداد مقادیر یکتا در Team 1:", df["Team 1"].nunique())
print("تعداد مقادیر یکتا در Team 2:", df["Team 2"].nunique())
print("تعداد مقادیر یکتا در Winner:", df["Winner"].nunique())

print("\nمقدارهای یکتا در Team 1:", df["Team 1"].unique())
print("\nمقدارهای یکتا در Team 2:", df["Team 2"].unique())
print("\nمقدارهای یکتا در Winner:", df["Winner"].unique())


تعداد مقادیر یکتا در Team 1: 78
تعداد مقادیر یکتا در Team 2: 83
تعداد مقادیر یکتا در Winner: 65

مقدارهای یکتا در Team 1: ['France' 'USA' 'Yugoslavia' 'Romania' 'Argentina' 'Chile' 'Uruguay'
 'Brazil' 'Paraguay' 'Austria' 'Hungary' 'Switzerland' 'Sweden' 'Germany'
 'Spain' 'Italy' 'Czechoslovakia' 'Cuba' 'England' 'Germany FR' 'Turkey'
 'Soviet Union' 'Northern Ireland' 'Mexico' 'Wales' 'Portugal' 'Korea DPR'
 'Peru' 'Belgium' 'Bulgaria' 'German DR' 'Zaire' 'Poland' 'Australia'
 'Scotland' 'Netherlands' 'Haiti' 'Tunisia' 'Algeria' 'Honduras' 'Canada'
 'Morocco' 'Korea Republic' 'Iraq' 'Denmark' 'United Arab Emirates'
 'Costa Rica' 'Cameroon' 'Republic of Ireland' 'Colombia' 'Norway'
 'Nigeria' 'Saudi Arabia' 'Bolivia' 'Russia' 'Greece' 'Jamaica'
 'South Africa' 'Japan' 'Croatia' 'China PR' 'Senegal' 'Slovenia'
 'Ecuador' 'Trinidad and Tobago' 'Serbia and Montenegro' 'Angola'
 'Czech Republic' 'Togo' 'Iran' 'Ivory Coast' 'Ghana' 'Ukraine' 'Serbia'
 'New Zealand' 'Slovakia' 'IR Iran' 'Bo

In [20]:
from sklearn.preprocessing import LabelEncoder

team_encoder = LabelEncoder()

all_teams = list(set(df["Team 1"].unique()).union(set(df["Team 2"].unique())))

team_encoder.fit(all_teams)

df["Team 1 Encoded"] = team_encoder.transform(df["Team 1"])
df["Team 2 Encoded"] = team_encoder.transform(df["Team 2"])

winner_encoder = LabelEncoder()

df["Winner"] = df["Winner"].apply(lambda x: x if x in all_teams else "Draw")

winner_encoder.fit(df["Winner"])

df["Winner Encoded"] = winner_encoder.transform(df["Winner"])

features = ["Team 1 Encoded", "Team 2 Encoded", "Score 1", "Score 2"]
target = "Winner Encoded"

df[features + [target]].head()


,Team 1 Encoded,Team 2 Encoded,Score 1,Score 2,Winner Encoded
0,26,46,4,1,20
1,76,5,3,0,60
2,81,8,2,1,64
3,58,54,3,1,45
4,2,26,1,0,1


In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

print("ابعاد مجموعه آموزش:", X_train.shape, y_train.shape)
print("ابعاد مجموعه تست:", X_test.shape, y_test.shape)


ابعاد مجموعه آموزش: (681, 4) (681,)
ابعاد مجموعه تست: (171, 4) (171,)


In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"🎯 دقت مدل: {accuracy:.2%}")


🎯 دقت مدل: 70.18%


In [23]:
def predict_winner(team1, team2, score1=0, score2=0):
    if team1 not in team_encoder.classes_ or team2 not in team_encoder.classes_:
        return "❌ خطا: یکی از تیم‌ها در دیتاست موجود نیست."

    team1_encoded = team_encoder.transform([team1])[0]
    team2_encoded = team_encoder.transform([team2])[0]

    match_data = [[team1_encoded, team2_encoded, score1, score2]]

    predicted_winner_encoded = model.predict(match_data)[0]

    predicted_winner = winner_encoder.inverse_transform([predicted_winner_encoded])[0]

    return f"🏆 پیش‌بینی برنده: {predicted_winner}"

team1 = "Brazil"
team2 = "Argentina"
print(predict_winner(team1, team2))


🏆 پیش‌بینی برنده: Draw


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [25]:
import pandas as pd

def predict_winner(team1, team2, score1=0, score2=0):
    if team1 not in team_encoder.classes_ or team2 not in team_encoder.classes_:
        return "❌ خطا: یکی از تیم‌ها در دیتاست موجود نیست."

    team1_encoded = team_encoder.transform([team1])[0]
    team2_encoded = team_encoder.transform([team2])[0]

    match_data = pd.DataFrame([[team1_encoded, team2_encoded, score1, score2]], columns=features)

    predicted_winner_encoded = model.predict(match_data)[0]

    predicted_winner = winner_encoder.inverse_transform([predicted_winner_encoded])[0]

    return f"🏆 پیش‌بینی برنده: {predicted_winner}"

print(predict_winner("Brazil", "Argentina"))


🏆 پیش‌بینی برنده: Argentina


In [26]:
team1 = input("🏆 نام تیم اول را وارد کنید: ")
team2 = input("🏆 نام تیم دوم را وارد کنید: ")

if team1 in team_encoder.classes_ and team2 in team_encoder.classes_:
    result = predict_winner(team1, team2)
    print(result)
else:
    print("❌ خطا: یکی از تیم‌ها در دیتاست موجود نیست. لطفاً نام تیم را درست وارد کنید.")


🏆 نام تیم اول را وارد کنید: France
🏆 نام تیم دوم را وارد کنید: Mexico
🏆 پیش‌بینی برنده: France
